# Time series - recurrent network text

In this example, we will rpedict a very simple artificial time series. We can look at it as a regression in time.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras import Model
from keras.layers import Input, Dense, Dropout, LSTM, SimpleRNN

%matplotlib inline

This is what we are going to predict: A cumulative sum of a random input value. Here's how it looks like:

$
\begin{align}
y_t = tanh(\sum_{i=1}^t x_i)
\end{align}
$

Hyperbolic tangent is there just for keeping it in a reasonable output interval.

In [ ]:
samples = np.random.uniform(low=-0.1, high=0.1, size=(2000))

plt.figure(figsize=(20,5))
plt.plot(samples)
plt.plot(np.tanh(np.cumsum(samples)))
plt.show()

Next, we are going to prepare a training set. We'll sample the same function a few thousand times:

In [ ]:
NUM_SAMPLES = 2000
SEQ_LEN = 200

raw_train_X = [np.random.uniform(low=-0.1, high=0.1, size=(SEQ_LEN)) for i in range(NUM_SAMPLES)]
raw_train_Y = [np.tanh(np.cumsum(x)) for x in raw_train_X]

Now, let's build a simple feed-forward network for prediction of this series. Notice that it has to have the whole sequence on the input.

In [ ]:
inputs = Input(shape=(SEQ_LEN,))
x = Dense(200, activation='tanh')(inputs)
x = Dense(200, activation='tanh')(x)
x = Dense(200, activation='tanh')(x)
outputs = Dense(SEQ_LEN, activation='tanh')(x)

ffn_model = Model(inputs, outputs)
ffn_model.compile(optimizer='adam', loss='mse')
ffn_model.summary()

train_X = np.vstack(raw_train_X)
train_Y = np.vstack(raw_train_Y)

Now we can fit the training data:

In [ ]:
progress = ffn_model.fit(train_X, train_Y, validation_split=0.1, epochs=25)

Let's test out the model on newly generated sequence.

In [ ]:
test_sample = np.random.uniform(low=-0.1, high=0.1, size=(SEQ_LEN))

Or make some non-standard test sample...

In [ ]:
test_sample = np.concatenate([np.zeros(50), np.random.uniform(low=-1, high=1, size=(SEQ_LEN - 50))])

Let's see how our model performs on a `test_sample`

In [ ]:
prediction = ffn_model.predict(test_sample.reshape((1, SEQ_LEN)))[0]

plt.figure(figsize=(20,5))
plt.plot(test_sample)
plt.plot(np.tanh(np.cumsum(test_sample)), 'r--')
plt.plot(prediction, 'g-')
plt.show()

Finally let's try some recurrent networks:

In [ ]:
inputs = Input(shape=(None, 1))
x = SimpleRNN(32, return_sequences=True)(inputs)
x = SimpleRNN(32, return_sequences=True)(x)
outputs = SimpleRNN(1, return_sequences=True)(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='mse')
model.summary()

train_X = np.vstack(raw_train_X).reshape((NUM_SAMPLES, SEQ_LEN, 1))
train_Y = np.vstack(raw_train_Y).reshape((NUM_SAMPLES, SEQ_LEN, 1))

progress = model.fit(train_X, train_Y, validation_split=0.1, epochs=25)

In [ ]:
inputs = Input(shape=(None, 1))
x = LSTM(32, return_sequences=True)(inputs)
outputs = Dense(1, activation='tanh')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='mse')
model.summary()

train_X = np.vstack(raw_train_X).reshape((NUM_SAMPLES, SEQ_LEN, 1))
train_Y = np.vstack(raw_train_Y).reshape((NUM_SAMPLES, SEQ_LEN, 1))

progress = model.fit(train_X, train_Y, validation_split=0.1, epochs=25)

In [ ]:
TEST_SEQ_LEN = 500
test_sample = np.random.uniform(low=-0.1, high=0.1, size=(TEST_SEQ_LEN))

prediction = model.predict(test_sample.reshape((1, TEST_SEQ_LEN, 1)))[0]

plt.figure(figsize=(20,5))
plt.plot(test_sample)
plt.plot(np.tanh(np.cumsum(test_sample)), 'r--')
plt.plot(prediction, 'g-')
plt.show()

In [ ]:
#utility code to plot loss history

loss_history = np.array(progress.history['loss'])
val_loss_history = np.array(progress.history['val_loss'])
                            
plt.figure(figsize=(20,5))
plt.ylim(ymin=0, ymax=max(np.max(loss_history), np.max(val_loss_history)))
plt.plot(loss_history, 'r-')
plt.plot(val_loss_history, 'g-')
plt.show()